## Module loading

In [1]:
from importlib import reload

import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

from lib.data import tools
reload(tools)
from lib.data.tools import get_dataloader
from lib.train.runners import PytorchRunner
from lib.train.metrics import IoUMetricMeter
from models.modelInterface import BDD100kModel
import lib.simulation.env
reload(lib.simulation.env)
from lib.simulation.env import (get_image_paths, get_label_paths, get_transforms)

## Set parameters

In [2]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pkg_name = "10k" # 100k or 10k
output_size = (512,1024)
task_name = "sem_seg" # "drivable", "sem_seg"
config_file = "/home/zekun/drivable/src/models/config-deeplabv3plus-sem_seg.py"
checkpoint_file = "/home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_backbone-centralized.pth"
new_checkpoint_file = "/home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_backbone-centralized.pth"
train_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_train.json"
val_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_val.json"
output_dir = "/home/zekun/drivable/outputs/semantic"

batchsize = 4
learn_rate = 0.0001
epochs = 10
num_workers = 0

In [4]:
IMAGE_PATH, IMAGE_PATH_TRAIN, IMAGE_PATH_VAL = get_image_paths("/home/zekun/drivable/", pkg_name)
LABEL_PATH, LABEL_PATH_TRAIN, LABEL_PATH_VAL = get_label_paths("/home/zekun/drivable/", task_name, pkg_name)
img_transform, lbl_transform = get_transforms(task_name, output_size)
conditions = []

classes_num = 20

## Data loaders

In [20]:
train_fns = tools.get_img_paths_by_conditions(conditions, train_attr_file, IMAGE_PATH_TRAIN)
val_fns = tools.get_img_paths_by_conditions(conditions, val_attr_file, IMAGE_PATH_VAL)

train_loader = get_dataloader(
    train_fns,
    batch_size=batchsize,
    workers=num_workers,
    img_transform=img_transform,
    lbl_transform=lbl_transform,
    img_path=IMAGE_PATH_TRAIN,
    lbl_path=LABEL_PATH_TRAIN,
    is_train=True,
    classes_num=classes_num
)
val_loader = get_dataloader(
    val_fns,
    batch_size=batchsize,
    workers=num_workers,
    img_transform=img_transform,
    lbl_transform=lbl_transform,
    img_path=IMAGE_PATH_VAL,
    lbl_path=LABEL_PATH_VAL,        
    is_train=False,
    classes_num=classes_num
)

## Model initialization

In [ ]:
model = BDD100kModel(
    num_classes=classes_num,
    backbone=tools.load_mmcv_checkpoint(config_file, checkpoint_file),
    size=output_size
)
# tools.load_checkpoint(model, checkpoint_file)
# tools.save_model(model.backbone, checkpoint_file)

optimizer = optim.Adam(model.parameters(), lr=learn_rate)
criterion = nn.CrossEntropyLoss(ignore_index=255)

runner = PytorchRunner(optimizer, criterion, train_loader, val_loader, IoUMetricMeter(classes_num), DEVICE, True)

## Model training

In [23]:
runner.train(model, epochs)

Train: 100%|██████████| 1750/1750 [18:40<00:00,  1.56it/s, loss=0.116, pAcc=0.96]
DEBUG flames 2023-10-12 23:42:25,730 | runners.py:180 | {'comment': 'training log', 'epoch': 0, 'log': {'loss': 0.1160520529044526, 'pAcc': 0.9603119171982196, 'Acc': [0.9849085974464687, 0.9026111041515957, 0.9484844237506895, 0.9163420208594651, 0.8986607443250397, 0.794760676744661, 0.8064973636955839, 0.8660948306306544, 0.9393969450647139, 0.8636094856782397, 0.9841600529232759, 0.8457618098992797, 0.7593067898583704, 0.9653147743043453, 0.9502226835445007, 0.9559209805032812, 0.9885593220338983, 0.7596373778665282, 0.8255937729375417], 'mIoU': 0.7384643951207274, 'IoU': [0.9708990601599815, 0.8147779216397106, 0.906324250624672, 0.8169263904125331, 0.8124311435464098, 0.584171857905339, 0.6220215531456228, 0.6996494681578452, 0.897714688926668, 0.7320578545245875, 0.9675052223930626, 0.7129550235802017, 0.5733764274940746, 0.9391214894730905, 0.9010028823175658, 0.9138031991985858, 0.004629969576753

KeyboardInterrupt: 

## Model Validation

In [24]:
valid_logs = runner.validate(model)
print(valid_logs)

Valid: 100%|██████████| 250/250 [00:56<00:00,  4.39it/s, loss=0.286, pAcc=0.932]
DEBUG flames 2023-10-13 00:03:16,272 | runners.py:194 | {'comment': 'validating log', 'log': {'loss': 0.28594414007663727, 'pAcc': 0.932371790869668, 'Acc': [0.9661183834632512, 0.8108512950796362, 0.9098675036155082, 0.49273669690301697, 0.6824299972609191, 0.7248313244919246, 0.784213741893035, 0.7497396403016247, 0.9350343388729282, 0.6227064857228221, 0.9755990970947657, 0.7991517946569333, 0.6502722537878788, 0.9453456735145814, 0.7983801077172566, 0.8907954606531314, 0.0, 0.45704347826086955, 0.5712433211262532], 'mIoU': 0.5859109607409825, 'IoU': [0.944570068024539, 0.6472956172912846, 0.8533903083420729, 0.3232128476242039, 0.4912860246472642, 0.5229177003546337, 0.5595600378930778, 0.5422852637529381, 0.8654785729685265, 0.4884687541955892, 0.9534308726826696, 0.6159531017137226, 0.34573689758799186, 0.9038844072224925, 0.5605701712167919, 0.7697341391903341, 0.0, 0.3417770905885583, 0.40275637878

{'loss': 0.28594414007663727, 'pAcc': 0.932371790869668, 'Acc': [0.9661183834632512, 0.8108512950796362, 0.9098675036155082, 0.49273669690301697, 0.6824299972609191, 0.7248313244919246, 0.784213741893035, 0.7497396403016247, 0.9350343388729282, 0.6227064857228221, 0.9755990970947657, 0.7991517946569333, 0.6502722537878788, 0.9453456735145814, 0.7983801077172566, 0.8907954606531314, 0.0, 0.45704347826086955, 0.5712433211262532], 'mIoU': 0.5859109607409825, 'IoU': [0.944570068024539, 0.6472956172912846, 0.8533903083420729, 0.3232128476242039, 0.4912860246472642, 0.5229177003546337, 0.5595600378930778, 0.5422852637529381, 0.8654785729685265, 0.4884687541955892, 0.9534308726826696, 0.6159531017137226, 0.34573689758799186, 0.9038844072224925, 0.5605701712167919, 0.7697341391903341, 0.0, 0.3417770905885583, 0.40275637878197856]}


In [25]:
tools.save_model(model.backbone, new_checkpoint_file, epochs, valid_logs['mIoU'])

model saved to /home/zekun/drivable/outputs/semantic/db/models/deeplabv3+_backbone-centralized.pth


In [61]:
for k,v in valid_logs.items():
    print(k, "=", v)

Loss = 0.31901394536147765
mAcc = 0.9128786127915733
Acc = [0.95043859 0.80964294 0.8643484  0.5441846  0.80412085 0.76782445
 0.8249825  0.82894086 0.87843651 0.55570706 0.97687499 0.86176194
 0.42660393 0.93757133 0.8047926  0.86462452 0.         0.46097626
 0.57384185]
mIoU = 0.5385917050450463
IoU = [0.9277337703984144, 0.5749089554192723, 0.8116293222447687, 0.11500538810019596, 0.42866139115677754, 0.4894062162248554, 0.5022291757045776, 0.5112494661005472, 0.8050132337244152, 0.3669570105627057, 0.9323787187552106, 0.5322123755980739, 0.324821413015071, 0.8794332374533477, 0.5546674134801459, 0.7573088192617695, 0.0, 0.2895744612381065, 0.4300520274176233]


## Appendix

In [64]:
x1 = [1,1,1]
x2 = [1,1,2]

def calculate_iou(pred_mask, true_mask, class_label):
    pred_class = pred_mask == class_label
    true_class = true_mask == class_label

    intersection = np.logical_and(pred_class, true_class)
    union = np.logical_or(pred_class, true_class)

    iou_score = np.sum(intersection) / (np.sum(union) + 0.00005)
    return iou_score

def calculate_mean_iou(pred_mask, true_mask, num_classes):
    miou_sum = 0.0
    for class_label in range(num_classes):
        iou = calculate_iou(pred_mask, true_mask, class_label)
        print(iou)
        miou_sum += iou

    mean_iou = miou_sum / num_classes
    return mean_iou

num_classes = 3
miou = calculate_mean_iou(np.array([x1, x1]), np.array([x2, x1]), num_classes)
print("mIoU:", miou)
print(x1[0])

0.0
0.8333263889467588
0.0
mIoU: 0.2777754629822529
1
